# Building a Workflow for the NYU Retinotopy Dataset

## About this Notebook

This notebook is an example of how you can construct an entire preprocessing pipeline for an fMRI dataset of a visual experiment. We use the NYU Retinotopy Dataset ([paper](https://doi.org/10.1016/j.neuroimage.2021.118609), [dataset](https://openneuro.org/datasets/ds003787/)), in which participants participated in retinotopic mapping scans, as an example dataset.

This notebook was made for the 2024 VSS Workshop [*How to stop worrying and love neuroimaging of the visual cortex*](https://noahbenson.github.io/vss2024), and it is specifically designed to run seamlessly inside of a [Neurodesk](https://neurodesk.org/) instance. Neurodesk instances can be run in the cloud, on high performance compute clusters (using singularity), or by using [Docker](https://docker.com/). For information on running Neurodesk locally for use with this notebook, see the README file of the [GitHub repository](https://github.com/noahbenson/vss2024) associated with the workshop.

Part of what makes this notebook useful is that it serves as an example of a complete reproducible preprocessing pipeline. All of the steps in this notebook use open source software whose versions are noted in the notebook itself. Because the notebook is designed to run in Neurodesk, many of the issues related to software versions and installation are eliminated. The [GitHub repository for the workshop](https://github.com/noahbenson/vss2024) additionally demonstrates how to use Docker and `docker-compose` to allow others to access the same environment in which you run these preprocessing steps.

This tutorial is by [Noah C. Benson](https://nben.net/) (eScience Institute, University of Washington). Please email [nben@uw.edu](mailto:nben@uw.edu) if you have any questions.

### Notebook Organization and Goals


The notebook is organized into steps, which, when run in order in a clean Neurodesk instance, should consist of an entire preprocessing workflow for a single subject. Note that some of the steps, such as running `fMRIprep`, produce a lot of output. **If you are running this notebook outside of the Neurodesk virtual compute instance set up for the VSS 2024 workshop, then you will need to follow the instructions below on obtaining the dataset**. Alternatively, if you have your own dataset that you wish to analyze, you can upload it and preprocess it instead (doing so will require changing some of the settings and commands below).

The goal of the workflow in this notebook is to preprocess the data of a single subject of the NYU Retinotopy Dataset (though only minor changes are needed to the notebook to enable it to preprocess the entire dataset). We begin the workflow with the assumption that the dataset is already in BIDS format, but we then complete all of the following steps:

1. **Preprocess the dataset using fMRIprep**. This step performs most major preprocessing steps, including fieldmap unwarping, slice time correction, skull stripping, gray-/white-matter segmentation, image normalization, and all cortical surface modeling steps performed by FreeSurfer.
2. **Solve population receptive field (PRF) models models**. This step converts the BOLD timeseries (measured during the retinotopic mapping scans) from each point on the cortical surface into estimates of the PRF center (*x*, *y* in degrees of the visual field), PRF size (*\u03C3*, in degrees of the visual field), and the fraction of variance from the timeseries that is explained by the PRF model.
3. **Visualize the retinotopic maps and label visual areas**. This step annotates which regions of cortex are part of V1, V2, V3, hV4, and a number of other visual areas.

### About the Dataset

The NYU Retinotopy Dataset ([paper](https://doi.org/10.1016/j.neuroimage.2021.118609), [dataset](https://openneuro.org/datasets/ds003787/)) was originally collected with the intention of verifying the reproducibility of various observations about human vision that came out of the much larger Human Connectome Project 7 Tesla Retinotopy Dataset ([paper](https://doi.org/10.1167/18.13.23), [dataset](https://osf.io/bw9ec/)). Subjects participated in at least two T1-weighted scans and at least six retinotopic mapping scans (192 seconds each) consisting of drifting bars containing colorful images (similar to the stimulus used in the HCP protocol, but with only bars instead of bars, rings, and wedges).

#### Obtaining the Dataset
**You can skip these steps if you are running this notebook in the virtual Neurodesk instance set up for the VSS Workshop.** During the workshop, the NYU Retinotopy Dataset will be downloaded ahead of time and will be available in the directory `/home/jovyan/shared/lec-02/data/NYU_full`. If you are running this notebook in another environment (such as on your own computer), you will need to make sure that the dataset is downloaded ahead of time. To ensure the dataset is downloaded, you will need to run the following commands (you will need to copy-paste these into new cells and run them).

First, run the following shell-commands to install tools for downloading data from OpenNeuro. You will need to copy-and-paste these code blocks into Jupyter cells to run them:
```bash
%%bash

# First, we need to install cloudpathlib, which lets us download data
# from AWS S3 service, where OpenNeuro datasets are stored.
pip install 'cloudpathlib[s3] == 0.18.1'

# Next, we need to make a directory for our dataset. We use the same
# directory that was used in the VSS workshop so that we don't need to
# change the paths in this notebook.
mkdir -p /home/jovyan/shared/lec-02/data/step0_raw
# We also need a directory for the stimulus files.
mkdir -p /home/jovyan/shared/lec-02/data/stimulus
```

Then, run this Python code in a separate cell to download the dataset. Note that this will take quite some time to run (this is normal):

```python
from cloudpathlib import S3Path, S3Client
from pathlib import Path

data_path = S3Path(
    's3://openneuro.org/ds003787/',
    client=S3Client(no_sign_request=True))
dest_path = Path(
    '/home/jovyan/shared/lec-02/data/step0_raw')
print("Downloading files:")
print("Downloading files for sub-wlsubj001...")
subpath = data_path / 'sub-wlsubj001'
subpath.download_to(dest_path / 'sub-wlsubj001')
# Also download the stimulus (all subjects / sessions are the same):
stim_path = data_path / 'derivatives' / 'stimulus_apertures'
stim_path = stim_path / 'sub-wlsubj001' / 'ses-nyu3t01'
stim_path = stim_path / 'sub-wlsubj001_ses-nyu3t01_task-prf_apertures.nii.gz'
stim_path.download_to(
    '/home/jovyan/shared/lec-02/data/stimulus/task-prf_apertures.nii.gz')
```

Finally, you will need to upload a license file for FreeSurfer to your home directory and name it `fslicense`. This is only needed if you intend to run fMRIprep. You can upload the file by dragging it into the file browser menu of JupyterLab while you are viewing your home directory. More information on obtaining a license file is below in the first code cell.

## Configuration

We begin the workflow by configuring a few meta-data items. These pieces of data largely tell the workflow where to find various data. We set these variables in the `os.environ` dictionary, which represents the Operating System's environment for the currently running notebook. The OS environment is a dictionary of strings that is inherited by any process or command that is started by the notebook, so if we run a bash script or a FreeSurfer command from this notebook, then that script or command will also inherit these values. See the [Wikipedia page on Environment Variables](https://en.wikipedia.org/wiki/Environment_variable) for more information.

In [1]:
from os import environ

# The path where we can find the raw MRI scans in BIDS format.
environ['RAWDATA_DIR'] = '/home/jovyan/shared/lec-02/data/step0_raw'

# The work directory in which we make and run the workflow.
environ['WORK_DIR'] = '/home/jovyan/work'

# The output path, in which the preprocessed data will be stored in BIDS format.
environ['PREPROC_DIR'] = f'{environ["WORK_DIR"]}/BIDS'

# The file that contains the stimulus apertures used in the experiment.
environ['STIMULUS_FILE'] = '/home/jovyan/shared/lec-02/data/stimulus/task-prf_apertures.nii.gz'

# To run fMRIprep using FreeSurfer, you need a license from FreeSurfer. These
# are free. See https://surfer.nmr.mgh.harvard.edu/fswiki/License for information.
# This should be the path to that license file.
environ['FS_LICENSE'] = '/home/jovyan/fslicense'

# The number of threads to run.
environ['ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS'] = '4'

# This fixes a warning from matplotlib when fMRIprep starts up in Neurodesk.
environ['MPLCONFIGDIR'] = '/home/jovyan/.matplotlib-mpldir'

Next, make sure that we're in the work directory.

In [2]:
from os import chdir

chdir(environ['WORK_DIR'])

We want to make sure that these directories exist before we run the rest of this notebook.

In [3]:
%%bash

mkdir -p BIDS
mkdir -p tmp
mkdir -p config

## Step 1. Run fMRIprep

The first step in preprocessing the data for this project is to use `fMRIprep` to perform most of the basic preprocessing. 
We will preprocess this dataset using the `fMRIprep` version noted in the **Configuration** section above.

### Load the fMRIprep module

In [4]:
# Load the fMRIprep module.
import lmod
await lmod.load(f'fmriprep/23.2.1')

### Run the fMRIprep command

In [5]:
%%bash

# Run the fMRIprep command:
fmriprep "$RAWDATA_DIR" "$PREPROC_DIR" participant \
    --participant-label wlsubj001 \
    --output-spaces T1w fsnative \
    --bold2t1w-init header \
    --fs-license-file "$FS_LICENSE" \
    --nprocs "$ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS" \
    --mem 12000 \
    -w tmp/fmriprep \
    --skip_bids_validation \
    --verbose

You are using fMRIPrep-23.2.1, and a newer version of fMRIPrep is available: 23.2.2.
Please check out our documentation about how and when to upgrade:
https://fmriprep.readthedocs.io/en/latest/faq.html#upgrading


240513-19:53:30,805 cli INFO:
	 Telemetry system to collect crashes and errors is enabled - thanks for your feedback!. Use option ``--notrack`` to opt out.
240513-19:55:40,302 nipype.workflow IMPORTANT:
	 Running fMRIPrep version 23.2.1

         License NOTICE ##################################################
         fMRIPrep 23.2.1
         Copyright 2023 The NiPreps Developers.
         
         This product includes software developed by
         the NiPreps Community (https://nipreps.org/).
         
         Portions of this software were developed at the Department of
         Psychology at Stanford University, Stanford, CA, US.
         
         This software is also distributed as a Docker container image.
         The bootstrapping file for the image ("Dockerfile") is licensed
         under the MIT License.
         
         This software may be distributed through an add-on package called
         "Docker Wrapper" that is under the BSD 3-clause License.
         ######

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



240514-02:21:35,208 nipype.workflow INFO:
	 [Node] Executing "ds_report_compcor" <fmriprep.interfaces.DerivativesDataSink>
240514-02:21:35,227 nipype.workflow INFO:
	 [Node] Finished "ds_report_compcor", elapsed time 0.018596s.
240514-02:21:35,265 nipype.workflow INFO:
	 [Job 1693] Completed (fmriprep_23_2_wf.sub_wlsubj001_wf.bold_ses_nyu3t01_task_prf_acq_PA_run_07_wf.bold_confounds_wf.ds_report_compcor).
240514-02:21:35,507 nipype.workflow INFO:
	 [Node] Setting-up "fmriprep_23_2_wf.sub_wlsubj001_wf.bold_ses_nyu3t01_task_prf_acq_PA_run_07_wf.bold_confounds_wf.merge_confound_metadata" in "/home/jovyan/work/tmp/fmriprep/fmriprep_23_2_wf/sub_wlsubj001_wf/bold_ses_nyu3t01_task_prf_acq_PA_run_07_wf/bold_confounds_wf/merge_confound_metadata".
240514-02:21:35,660 nipype.workflow INFO:
	 [Node] Executing "merge_confound_metadata" <nipype.interfaces.utility.base.Merge>
240514-02:21:35,662 nipype.workflow INFO:
	 [Node] Finished "merge_confound_metadata", elapsed time 0.001364s.
240514-02:21:35

### Check the fMRIprep BOLD outputs

The `fMRIprep` command should have produced a variety of outputs, including the BOLD data projected onto the cortical surface. It also produces a variety of reports in html files that are placed in the BIDS directory. We can open these files by launching a Neurodesk Desktop instance then opening them in Firefox. To launch Neurodesktop, click on the blue plus in the upper left of the Jupyter interface, then select the "Neurodesktop" button in the first row.

## Step 2. Fit PRF Models

### Build the PRFModel singularity container

The PRFModel software is stored in a Docker container. In Neurodesk (or on a cluster) we need to start by turning this into a Singularity image.

This may take some time to run, but only needs to be run once.

In [6]:
%%bash

# Convert the docker-image of prfanalyze-vista into a singularity image.
singularity build \
    tmp/prfanalyze-vista_2.2.2_3.1.1.simg \
    docker://garikoitz/prfanalyze-vista:2.2.2_3.1.1

INFO:    Starting build...
Getting image source signatures
Copying blob sha256:f4faa6ced19cd086ef67e7b7e97e29a0a9d27bcaeb5d642047bbc89589e13b30
Copying blob sha256:b51569e7c50720acf6860327847fe342a1afbe148d24c529fb81df105e3eed01
Copying blob sha256:fb15d46c38dcd1ea0b1990006c3366ecd10c79d374f341687eb2cb23a2c8672e
Copying blob sha256:58690f9b18fca6469a14da4e212c96849469f9b1be6661d2342a4bf01774aa50
Copying blob sha256:da8ef40b9ecabc2679fe2419957220c0272a965c5cf7e0269fa1aeeb8c56f2e1
Copying blob sha256:d46ab28b2b2efbb2df9f89997969b925585d9dab39e6124e99e6bd8c801b839a
Copying blob sha256:78052fda4ba730b35950073437798355b832e63196ae3fc86dcaa1f01ebfdc23
Copying blob sha256:37c9f603ecf526d7a12a14c24f278f64740353f54c959b3459e47a9763cffa90
Copying blob sha256:b4e8fc7b2f7176ad516ee16b1b5899adfa69dd77724d3cfdcf98ddba1ae7baa4
Copying blob sha256:b238860c41076fded809db31b775219a4ab6c197b0719faae8eb769d8583ccfd
Copying blob sha256:e53378506c1618c1e8583bcc0ca03980aedf228a30f65dbfc9b6cf118a060a01
Copyin

### Make a PRFModel configuration file

In order to run the PRFModel container, we need to provide it with a configuration JSON file. This
primarily contains information about the data that is to be fit and how to perform that fit. More
information can be found on the [PRFModel wiki](https://github.com/vistalab/PRFmodel/wiki). Because
PRFModel expects this to be called `config.json`, we write it out with this name (though we can
change it late).

In [7]:
%%writefile config.json
{
    "subjectName": "wlsubj001",
    "sessionName": "nyu3t01",
    "tasks": "prf",
    "isPRFSynthData": false,
    "solver": "vista",
    "options": {
        "prfprepareAnalysis": "01"
    },
    "stimulus": {
        "stimulus_diameter": 24
    }
}

Writing config.json


### Organize the BOLD data for PRFModel

#### The Stimulus Files

We also need to make sure that the stimulus information is available to PRFModel. PRFModel expects the
stimulus apertures to be stored in the `stimuli` subdirectory of the BIDS directory.

In [8]:
%%bash

FILENAME=sub-wlsubj001_ses-nyu3t01_task-prf_apertures.nii.gz
DIR=BIDS/derivatives/prfprepare/analysis-01/sub-wlsubj001/stimuli/

mkdir -p "$DIR"
cp "$STIMULUS_FILE" "$DIR/$FILENAME"

#### The BOLD files

In [9]:
%%bash

BASEDIR=BIDS/derivatives/prfprepare

mkdir -p $BASEDIR/analysis-01/sub-wlsubj001/ses-nyu3t01/func

In [10]:
import nibabel as nib
import numpy as np

subject = 'wlsubj001'
session = 'nyu3t01'
task = 'prf'

# Where we find the data and what it's called (and what we will rename it).
subdir_part = f'sub-{subject}/ses-{session}/func'
func_dir = f'BIDS/{subdir_part}'
prep_dir = f'BIDS/derivatives/prfprepare/analysis-01/{subdir_part}'
file_head = f'sub-{subject}_ses-{session}_task-{task}'

# We're going to convert the files for both hemispheres
for hemi in ['L', 'R']:
    # All the GIFTI files of a single hemisphere end like this:
    file_tail = f'hemi-{hemi}_space-fsnative_bold.func.gii'
    # We'll convert the files for these runs:
    for run in ['01']:
        gii_filename = f'{func_dir}/{file_head}_acq-PA_run-{run}_{file_tail}'
        nii_filename = f'{prep_dir}/{file_head}_run-{run}_hemi-{hemi}_bold.nii.gz'
        # Load in the GIFTI file.
        gii = nib.load(gii_filename)
        # Convert it into a NIFTI image.
        signals = np.transpose([row.data for row in gii.darrays])
        signals = signals[:,None,None,...]
        nii = nib.Nifti2Image(signals, np.eye(4))
        nii.header.set_xyzt_units('mm', 'sec')
        # Save it out to the directory.
        nib.save(nii, nii_filename)

We also need to make events files for the `prfprepare` directory. These files usually document when during experiments particular stimuli appear, but PRFModel just needs to know the stimulus file, so we can make these quite simple.

In [11]:
%%writefile BIDS/derivatives/prfprepare/analysis-01/sub-wlsubj001/ses-nyu3t01/func/sub-wlsubj001_ses-nyu3t01_task-prf_events.tsv
stim_file
sub-wlsubj001_ses-nyu3t01_task-prf_apertures.nii.gz

Writing BIDS/derivatives/prfprepare/analysis-01/sub-wlsubj001/ses-nyu3t01/func/sub-wlsubj001_ses-nyu3t01_task-prf_events.tsv


### Run the PRFModel Singularity Container

In [12]:
%%bash

# Clear the default singularity bind-paths.
export SINGULARITY_BINDPATH=""
export APPTAINER_BINDPATH=""

# Run the container with the bind-paths that prfmodel expects.
singularity run \
    --bind $PWD:/flywheel/v0/input \
    --bind $PWD:/flywheel/v0/output \
    tmp/prfanalyze-vista_2.2.2_3.1.1.simg

------------------------------------------
Setting up environment variables
---
LD_LIBRARY_PATH is .:/opt/mcr/v99//runtime/glnxa64:/opt/mcr/v99//bin/glnxa64:/opt/mcr/v99//sys/os/glnxa64:/opt/mcr/v99//sys/opengl/lib/glnxa64
This is the config.json file being read: /flywheel/v0/output/BIDS/derivatives/prfanalyze-vista/analysis-02/options.json
These are the contents of the json file:

tmp = 

  struct with fields:

    prfprepareAnalysis: '01'

/flywheel/v0/input/BIDS/derivatives/prfprepare/analysis-01/sub-wlsubj001/ses-nyu3t01/func/sub-wlsubj001_ses-nyu3t01_task-prf_run-01_hemi-L_bold.nii.gz
/tmp/tmpx9jo7u5r.json
/flywheel/v0/input/BIDS/derivatives/prfprepare/analysis-01/sub-wlsubj001/stimuli/sub-wlsubj001_ses-nyu3t01_task-prf_apertures.nii.gz
/flywheel/v0/output/BIDS/derivatives/prfanalyze-vista/analysis-02/options.json
--------------------------------------------------------------------------------
This is the options file comming in:

allOptions = 

  struct with fields:

    vista: [


real	189m19.456s
user	371m42.841s
sys	216m46.598s


------------------------------------------
Setting up environment variables
---
LD_LIBRARY_PATH is .:/opt/mcr/v99//runtime/glnxa64:/opt/mcr/v99//bin/glnxa64:/opt/mcr/v99//sys/os/glnxa64:/opt/mcr/v99//sys/opengl/lib/glnxa64
This is the config.json file being read: /flywheel/v0/output/BIDS/derivatives/prfanalyze-vista/analysis-02/options.json
These are the contents of the json file:

tmp = 

  struct with fields:

    prfprepareAnalysis: '01'

/flywheel/v0/input/BIDS/derivatives/prfprepare/analysis-01/sub-wlsubj001/ses-nyu3t01/func/sub-wlsubj001_ses-nyu3t01_task-prf_run-01_hemi-R_bold.nii.gz
/tmp/tmpf9ojzio7.json
/flywheel/v0/input/BIDS/derivatives/prfprepare/analysis-01/sub-wlsubj001/stimuli/sub-wlsubj001_ses-nyu3t01_task-prf_apertures.nii.gz
/flywheel/v0/output/BIDS/derivatives/prfanalyze-vista/analysis-02/options.json
--------------------------------------------------------------------------------
This is the options file comming in:

allOptions = 

  struct with fields:

    vista: [


real	172m29.247s
user	353m19.768s
sys	260m37.584s


Preparing solver "prfanalyze-vista":
[base/run.py] Using real data, not coming from prfsynthesize.
  Subject: wlsubj001
  Session: nyu3t01
  Options: {'prfprepareAnalysis': '01'}
These are the tasks: ['prf']
Output BIDS directory: /flywheel/v0/output/BIDS/derivatives/prfanalyze-vista/analysis-02/sub-wlsubj001/ses-nyu3t01
task used in the glob command is prf
['/flywheel/v0/input/BIDS/derivatives/prfprepare/analysis-01/sub-wlsubj001/ses-nyu3t01/func/sub-wlsubj001_ses-nyu3t01_task-prf_run-01_hemi-R_bold.nii.gz', '/flywheel/v0/input/BIDS/derivatives/prfprepare/analysis-01/sub-wlsubj001/ses-nyu3t01/func/sub-wlsubj001_ses-nyu3t01_task-prf_run-01_hemi-L_bold.nii.gz']
2
[base/run.py] This is the temp file with stim info: 
/tmp/tmpx9jo7u5r.json
[base/run.py] This is the content: 
{'stimulus_diameter': 24, 'isPRFSynthData': False}
Beginning os.wait() for /solve.sh, run=01 (child pid is 65814)
Processing estimates.json file...
Writing the estimates.json to nifti2 in outbids_dir: /flywheel/v0/outp

## Step 3. Visualize and Annotate the Retinotopic Maps

The final step in our preprocessing is to visualize the maps and delineate the boundaries of the visual areas in the occipital cortex. There are many tools for doing this including AFNI and VistaSoft, but all have substantial limitations. We will use a tool called [`cortex-annotate`](https://github.com/noahbenson/cortex-annotate), which was designed with the goals of being highly flexible and of allowing many raters to individually draw then merge annotations of a dataset using GitHub. 

Typically one has to configure `cortex-annotate` to work with a dataset (substantial documentation about this can be found on the `cortex-annotate` GitHub pate), but because we are using the output of `PRFModel`, we can use a preconfigured interface.

First, we install the package.

In [13]:
%%bash

pip install 'cortexannotate == 0.1.4'

  Attempting uninstall: cortexannotate
    Found existing installation: cortexannotate 0.1.3
    Uninstalling cortexannotate-0.1.3:
      Successfully uninstalled cortexannotate-0.1.3


Finally, we can run the `cortex-annotate` and annotate the maps!

In [ ]:
from cortexannotate import annotate_prfs

tool = annotate_prfs(
    'BIDS',
    prf_format='prfanalyze-vista',
    derivative='cortexannotate',
    overwrite=True,
    invert_y=True)